## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Carrega dados

In [2]:
filmes = pd.read_csv("movie_lens/movies.csv")

filmes.columns = ['filmeId', 'titulo', 'genero']
filmes.set_index('filmeId', inplace=True)
filmes.head()

,titulo,genero
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
notas = pd.read_csv("movie_lens/ratings.csv")
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


## Primeira tentativa de recomendação

In [5]:
total_de_votos = notas.filmeId.value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [6]:
filmes.loc[356]

titulo         Forrest Gump (1994)
genero    Comedy|Drama|Romance|War
Name: 356, dtype: object

In [7]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,titulo,genero,total_de_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [8]:
filmes.sort_values("total_de_votos", ascending=False).head(10)

,titulo,genero,total_de_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


In [9]:
notas_medias = notas.groupby('filmeId').mean().nota
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

## Uma segunda heurística

In [10]:
# Recomendar pelas notas medias mais altas
filmes['nota_media'] = notas_medias
filmes.sort_values("nota_media", ascending=False).head(10)

,titulo,genero,total_de_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [11]:
# filmes com mais de 10 votos
filmes.query("total_de_votos >= 10").sort_values("nota_media", ascending=False).head(10)


,titulo,genero,total_de_votos,nota_media
filmeId,,,,
1041,Secrets & Lies (1996),Drama,11.0,4.590909
3451,Guess Who's Coming to Dinner (1967),Drama,11.0,4.545455
1178,Paths of Glory (1957),Drama|War,12.0,4.541667
1104,"Streetcar Named Desire, A (1951)",Drama,20.0,4.475000
2360,"Celebration, The (Festen) (1998)",Drama,12.0,4.458333
1217,Ran (1985),Drama|War,15.0,4.433333
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
951,His Girl Friday (1940),Comedy|Romance,14.0,4.392857
1927,All Quiet on the Western Front (1930),Action|Drama|War,10.0,4.350000


In [12]:
# filmes com mais de 50 votos
filmes_com_mais_de_50_votos = filmes.query("total_de_votos >= 50")
filmes_com_mais_de_50_votos.sort_values("nota_media", ascending=False).head(10)


,titulo,genero,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [13]:
assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[assisti]

,titulo,genero,total_de_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [14]:
# Recomenda filmes baseado no genero de Jumanji
jumanji_generos = 'Adventure|Children|Fantasy'
filmes_com_mais_de_50_votos.query(f"genero == '{jumanji_generos}'").sort_values("nota_media", ascending=False).head(10)


,titulo,genero,total_de_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


## Procurando usuários similares

In [15]:
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query(f"usuarioId == {usuario}")
    return notas_do_usuario[['filmeId', 'nota']].set_index('filmeId')


In [16]:
usuario_1 = notas_do_usuario(1)
usuario_4 = notas_do_usuario(4)

In [17]:
diferencas = usuario_1.join(usuario_4, lsuffix="_esquerda", rsuffix="_direita").dropna()
diferencas.head()

,nota_esquerda,nota_direita
filmeId,,
47,5.0,2.0
235,4.0,2.0
260,5.0,5.0
296,3.0,1.0
441,4.0,1.0


In [18]:
def distancia_de_vetores(a, b):
    return np.linalg.norm(a-b)

In [19]:
distancia_de_vetores(diferencas.nota_esquerda, diferencas.nota_direita)

11.135528725660043

In [26]:
def distancia_de_usuarios(user_id1, user_id2):
    notas1 = notas_do_usuario(user_id1)
    notas2 = notas_do_usuario(user_id2)
    
    diferencas = notas1.join(notas2, lsuffix="_esquerda", rsuffix="_direita").dropna()
    distancia = distancia_de_vetores(diferencas.nota_esquerda, diferencas.nota_direita)
    return user_id1, user_id2, distancia

In [27]:
distancia_de_usuarios(1, 4)

(1, 4, 11.135528725660043)

## Distancia entre usuários do dataset

In [36]:
def distancia_de_todos(voce_id):
    usuarios = notas.usuarioId.unique()
    distancias = [distancia_de_usuarios(1, usuario) for usuario in usuarios]
    return pd.DataFrame(distancias, columns = ['voce', 'outra_pessoa', 'distancia'])



In [38]:
print(distancia_de_todos(1).head())

   voce  outra_pessoa  distancia
0     1             1   0.000000
1     1             2   1.414214
2     1             3   8.200610
3     1             4  11.135529
4     1             5   3.741657


## Usuários sem nad em comum são colocados bem distante um do outro

In [40]:
def distancia_de_usuarios(user_id1, user_id2, minimo=5):
    notas1 = notas_do_usuario(user_id1)
    notas2 = notas_do_usuario(user_id2)
    
    diferencas = notas1.join(notas2, lsuffix="_esquerda", rsuffix="_direita").dropna()
    
    if len(diferencas) < minimo:
        return (user_id1, user_id2, 1000000)
    
    distancia = distancia_de_vetores(diferencas.nota_esquerda, diferencas.nota_direita)
    return user_id1, user_id2, distancia

In [41]:
distancia_de_todos(1).head()

,voce,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,1000000.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


In [48]:
def mais_proximos_de_voce(voce):
    distancias = distancia_de_todos(voce)
    return distancias.sort_values("distancia").set_index("outra_pessoa").drop(voce)

In [49]:
mais_proximos_de_voce(1)

,voce,distancia
outra_pessoa,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
523,1,1.000000
49,1,1.000000
9,1,1.000000
258,1,1.000000
319,1,1.118034
398,1,1.224745


## Parametros para teste

In [54]:
# retorna somente n usuários
def distancia_de_todos(voce_id, n = None):
    usuarios = notas.usuarioId.unique()
    if n:
        usuarios = usuarios[:n]
    
    distancias = [distancia_de_usuarios(1, usuario) for usuario in usuarios]
    return pd.DataFrame(distancias, columns = ['voce', 'outra_pessoa', 'distancia'])



In [55]:
# retorna somente N usuários
def mais_proximos_de_voce(voce, n=None):
    distancias = distancia_de_todos(voce, n)
    return distancias.sort_values("distancia").set_index("outra_pessoa").drop(voce)

In [52]:
# Retorna 50 usuários
mais_proximos_de_voce(1, 50)

,voce,distancia
outra_pessoa,,
49,1,1.000000
9,1,1.000000
13,1,1.414214
25,1,1.414214
30,1,1.802776
35,1,2.236068
26,1,2.236068
46,1,3.316625
8,1,3.741657


In [56]:
# se o usuário não tem muitos filmes em comum retorna None

def distancia_de_usuarios(user_id1, user_id2, minimo=5):
    notas1 = notas_do_usuario(user_id1)
    notas2 = notas_do_usuario(user_id2)
    
    diferencas = notas1.join(notas2, lsuffix="_esquerda", rsuffix="_direita").dropna()
    
    if len(diferencas) < minimo:
        return None
    
    distancia = distancia_de_vetores(diferencas.nota_esquerda, diferencas.nota_direita)
    return user_id1, user_id2, distancia

In [57]:
# filtra para trazer somente os usuários que tem filmes em comum

def distancia_de_todos(voce_id, n = None):
    usuarios = notas.usuarioId.unique()
    if n:
        usuarios = usuarios[:n]
    
    distancias = [distancia_de_usuarios(1, usuario) for usuario in usuarios]
    distancias = list(filter(None,distancias)) # filtro
    return pd.DataFrame(distancias, columns = ['voce', 'outra_pessoa', 'distancia'])



In [59]:
# 50 usuarios (somente os que tem filmes em comum)
mais_proximos_de_voce(1, 50)

,voce,distancia
outra_pessoa,,
49,1,1.000000
9,1,1.000000
25,1,1.414214
13,1,1.414214
30,1,1.802776
35,1,2.236068
26,1,2.236068
46,1,3.316625
8,1,3.741657


In [106]:
# sugere filmes
def sugere_para(usuario, n = None):

    notas_de_usuario = notas_do_usuario(usuario)
    filmes_usuario_ja_viu = notas_de_usuario.index

    # 50 usuários
    similares = mais_proximos_de_voce(usuario, n)

    # usuário mais similar
    similar = similares.iloc[0].name

    notas_do_similar = notas_do_usuario(similar)
    # notas_do_similar
    notas_do_similar = notas_do_similar.drop(filmes_usuario_ja_viu, 
                                             errors='ignore')

    recomendacoes = notas_do_similar.sort_values("nota", ascending=True)

    return recomendacoes.join(filmes)[['titulo', 'genero', 'nota_media']]

In [107]:
# Sugere filmes para usuario 1
sugere_para(1).head()

,titulo,genero,nota_media
filmeId,,,
4878,Donnie Darko (2001),Drama|Mystery|Sci-Fi|Thriller,3.981651
5989,Catch Me If You Can (2002),Crime|Drama,3.921739
7438,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller,3.868182
7361,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,4.160305
4226,Memento (2000),Mystery|Thriller,4.122642


## Sugerindo baseado em vários usuários

In [118]:
# retorna N usuários mais próximos
def mais_proximos_de_voce(voce, n_mais_proximos=10, n_analisados=None):
    distancias = distancia_de_todos(voce, n_analisados)
    distancias = distancias.sort_values("distancia").set_index("outra_pessoa").drop(voce)
    return distancias.head(n_mais_proximos)

In [122]:
# K usuários mais próximos
def knn(usuario_id, k_mais_proximos=10, n_analisados=None):
    distancias = distancia_de_todos(usuario_id, n_analisados)
    distancias = distancias.sort_values("distancia")
    distancias = distancias.set_index("outra_pessoa").drop(usuario_id)
    return distancias.head(k_mais_proximos)

In [125]:
# sugere filmes usando n usuários mais proximos
def sugere_para(usuario, k_mais_proximos=10, n_analisados = None):
    notas_de_usuario = notas_do_usuario(usuario)
    filmes_usuario_ja_viu = notas_de_usuario.index

    # usuários mais proximos
    similares = knn(usuario, k_mais_proximos = k_mais_proximos, n_analisados = n_analisados)
    
    usuarios_similares = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuarios_similares]

    recomendacoes = notas_dos_similares.groupby("filmeId").mean()[['nota']]
    recomendacoes = recomendacoes.sort_values("nota", ascending=False)

    return recomendacoes.join(filmes)[['titulo', 'genero', 'nota_media']]

In [128]:
# sugere filmes para usuário 1 usando 10 vizinhos mais próximos
sugere_para(1, 10).head()

,titulo,genero,nota_media
filmeId,,,
1704,Good Will Hunting (1997),Drama|Romance,4.078014
57504,"Girl Who Leapt Through Time, The (Toki o kaker...",Animation|Comedy|Drama|Romance|Sci-Fi,4.100000
38304,No Direction Home: Bob Dylan (2005),Documentary,4.625000
48394,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller,3.814815
2300,"Producers, The (1968)",Comedy,3.969697
